In [1]:
!pip install --upgrade -q transformers datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


# Load and Analyze Data

In [3]:
# Load data
from datasets import load_dataset, DatasetDict


data = load_dataset('csv', data_files='/kaggle/input/contradictory-my-dear-watson/train.csv', split='train')
data = data.train_test_split(0.2)

data = DatasetDict({
    'train': data['train'],
    'val': data['test'],
    'test':load_dataset('csv', data_files='/kaggle/input/contradictory-my-dear-watson/test.csv', split='train')
})

data

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language'],
        num_rows: 5195
    })
})

In [4]:
# Since data is small we can analyze all of it using pandas dataframe
# if it was a big data we should have taken a random subsample of it using shuffle method

df = data['train'].to_pandas()
df

,id,premise,hypothesis,lang_abv,language,label
0,b624494c06,"One of them, darker skinned, had hair braided ...",They braid their hair to keep it out of the wa...,en,English,1
1,54536f82c6,العضو الثالث من الثالوث الهندوسي هو براهما، وم...,براهما هو جزء من الثالوث الهندوسي.,ar,Arabic,0
2,cae72ed01b,Kubadilika huko kwa hisia kulikuwa kwa ajabu.,Hisia ilibadilika sana.,sw,Swahili,0
3,cf0b006688,Если вы ищете кокосовый ром и прочие виды фрук...,К рому никогда не добавляются фруктовые вкусоа...,ru,Russian,2
4,bf2781e00b,Според централната група за информационна сигу...,Този процес обикновено се счита за вреден за с...,bg,Bulgarian,2
...,...,...,...,...,...,...
9691,92f93089aa,SSA is also seeking statutory authority for ad...,SSA wants the authority to recover underpayments.,en,English,2
9692,e2d5b58543,He was crying like his mother had just wallope...,He cried like his mom hit him.,en,English,0
9693,8b904ad06a,it it like strange that it you're right in the...,"You are in the middle of the mountains, but yo...",en,English,0
9694,435a9b88a7,China could never trump the warhead blizzard W...,China had many more nuclear warhead than the US.,en,English,2


In [5]:
df[['premise', 'hypothesis', 'lang_abv', 'language']].describe()

,premise,hypothesis,lang_abv,language
count,9696,9696,9696,9696
unique,7055,9695,15,15
top,"I understand, mademoiselle, I understand all y...",I am not sure.,en,English
freq,3,2,5512,5512


In [6]:
df[['lang_abv', 'language']].value_counts()

lang_abv  language  
en        English       5512
ar        Arabic         320
zh        Chinese        320
fr        French         318
sw        Swahili        315
ur        Urdu           309
el        Greek          305
ru        Russian        305
vi        Vietnamese     297
th        Thai           294
es        Spanish        290
hi        Hindi          288
tr        Turkish        283
de        German         279
bg        Bulgarian      261
Name: count, dtype: int64

In [7]:
df.isna().sum()

id            0
premise       0
hypothesis    0
lang_abv      0
language      0
label         0
dtype: int64

# Load Translation and Sentence Similarity Models

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

checkpoint = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
# checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, truncation=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

# Preprocess Data

In [9]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language'],
        num_rows: 5195
    })
})

In [10]:
def preprocess_data(batch):
    batch['text'] = [x1 + f' {tokenizer.sep_token} ' + x2 for x1,x2 in zip(batch['premise'],batch['hypothesis'])]
    return tokenizer(batch['text'])

data = data.map(preprocess_data, batched=True)
data

Map:   0%|          | 0/9696 [00:00<?, ? examples/s]

Map:   0%|          | 0/2424 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'text', 'input_ids', 'attention_mask'],
        num_rows: 5195
    })
})

# TrainerAPI

In [11]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_metric


def compute_metrics(eval_pred):
    metric = load_metric('accuracy')
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=labels)


train_args = TrainingArguments(
    output_dir='results',
    logging_dir='logs',
    report_to=[],
    
    evaluation_strategy='steps',
    eval_steps=200,
    logging_steps=200,
    save_steps=200,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.005,
    num_train_epochs=5,
    fp16=True,
    
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=3,
)

train_args

TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=200,
evaluation_strategy=steps,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_model

In [12]:
trainer = Trainer(
    model=model,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
200,0.322800,0.258479,0.904290
400,0.263900,0.289440,0.908828
600,0.202200,0.317681,0.898515
800,0.138800,0.366877,0.896865
1000,0.112700,0.506479,0.897277


/tmp/ipykernel_23/3248437400.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=1000, training_loss=0.20811964988708495, metrics={'train_runtime': 753.9471, 'train_samples_per_second': 64.302, 'train_steps_per_second': 2.009, 'total_flos': 1573418238013440.0, 'train_loss': 0.20811964988708495, 'epoch': 3.3})

# Predictions

In [13]:
preds = trainer.predict(data['test'])
preds

PredictionOutput(predictions=array([[-3.9796228, -2.220854 ,  6.036053 ],
       [-3.8100398,  5.862734 , -2.8288264],
       [ 6.6962047, -2.634575 , -2.5701075],
       ...,
       [ 6.519077 , -2.351316 , -2.7190206],
       [ 4.234051 , -1.3805493, -1.88628  ],
       [-4.183489 , -1.0506688,  4.994016 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 30.7062, 'test_samples_per_second': 169.184, 'test_steps_per_second': 5.308})

In [14]:
pd.DataFrame({
    'id': data['test']['id'],
    'prediction': np.argmax(preds.predictions, axis=1)
}).to_csv('submission.csv', header=True, index=False)